数据模型其实是对Python框架的描述，它规范了这门语言自身构建模块的接口，这些模块包括但不限于序列、迭代器、函数、类和上下文管理器。

In [1]:
import collections

In [2]:
Card = collections.namedtuple('Card', ['Rank', 'suit'])

In [3]:
Card

__main__.Card

In [5]:
class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

In [6]:
beer_card = Card('7', 'diamonds')

In [7]:
beer_card

Card(Rank='7', suit='diamonds')

In [8]:
deck = FrenchDeck()

In [9]:
len(deck)

52

In [70]:
deck[:2]

[Card(Rank='2', suit='spades'), Card(Rank='3', suit='spades')]

In [16]:
from random import choice

In [18]:
choice(deck)

Card(Rank='2', suit='hearts')

In [23]:
deck[12::13]

[Card(Rank='A', suit='spades'),
 Card(Rank='A', suit='diamonds'),
 Card(Rank='A', suit='clubs'),
 Card(Rank='A', suit='hearts')]

In [25]:
# for card in reversed(deck):
#     print(card)

In [27]:
deck[12::13][0] in deck

True

In [28]:
suit_values = dict(spades=3, hearts=2, diamonds=1, clubs=0)

In [86]:
def spades_high(card):
    rank_value = FrenchDeck.ranks.index(card.Rank)
    return rank_value * len(suit_values) + suit_values[card.suit]

In [87]:
len(sorted(deck, key=spades_high))

52

In [51]:
deck[0].suit

'spades'

## 1.2 如何使用特殊方法

特殊方法的存在是为了被Python解释器调用的，你自己并不需要调用它。特殊方法的刁难勇士隐式的，比如`for i in x:`这个语句，其实背后用的是`iter(x)`，而这个
函数的背后则是`x.__iter__()`方法。前提是这个方法在`x`对象中被实现过了。


In [52]:
from math import hypot

In [59]:
class Vector:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return "Vector(%r, %r)" % (self.x, self.y)
    
    def __abs__(self):
        return hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __add__(self, other):
        print("other:", other)
        x = self.x + other.x
        y = self.y + other.y
        return Vector(x, y)
    
    def __mul__(self, scalar):
        return Vector(self.x * scalar, self.y * scalar)

In [60]:
Vector(3, 4) + Vector(2, 3)

other: Vector(2, 3)


Vector(5, 7)

In [65]:
Vector(1, 2) * 3

Vector(3, 6)

In [66]:
abs(Vector(3, 4))

5.0

In [68]:
Vector('1', '2') + Vector('3', '4')

other: Vector('3', '4')


Vector('13', '24')

### 1.2.2 字符串的表示形式

Python有一个内置函数`repr`，它能把一个对象用字符串的形式表达出来以便辨认，这就是“字符串的表示形式”。`repr`就是通过`__repr__`这个特殊方法来得到一个对象的字符串表示形式的。

`__repr__` 和 `__str__` 的区别在于，后者是在 str() 函数被使用，或是在用 print 函数打印一个对象的时候才被调用的，并且它返回的字符串对终端用户更友好。如果你只想实现这两个特殊方法中的一个，__repr__ 是更好的选择，因为如果一个对象没有 __str__ 函数，而 Python 又需要调用它的时候，解释器会用 `__repr__`作为替代。



### 1.2.4 自定义的布尔值

默认情况下，我们自己定义的类的实例总被认为是真的，除非这个类对
__bool__ 或者 __len__ 函数有自己的实现。bool(x) 的背后是调用
x.__bool__() 的结果；如果不存在 __bool__ 方法，那么 bool(x) 会
尝试调用 x.__len__()。若返回 0，则 bool 会返回 False；否则返回
True。

如果想让 Vector.__bool__ 更高效，可以采用这种实现：

```
def __bool__(self):
    return bool(self.x or self.y)
```

## 1.4 为什么`len`不是普通方法

如果 x 是一个内置类
型的实例，那么 len(x) 的速度会非常快。背后的原因是 CPython 会直
接从一个 C 结构体里读取对象的长度，完全不会调用任何方法。获取一
个集合中元素的数量是一个很常见的操作，在
str、list、memoryview 等类型上，这个操作必须高效。

换句话说，len 之所以不是一个普通方法，是为了让 Python 自带的数据
结构可以走后门，abs 也是同理。但是多亏了它是特殊方法，我们也可
以把 len 用于自定义数据类型。这种处理方式在保持内置类型的效率和
保证语言的一致性之间找到了一个平衡点，也印证了“Python 之禅”中的
另外一句话：“不能让特例特殊到开始破坏既定规则。”